# Fluorescence decay node

Writing a factory that wraps  

In [22]:
import typing
import pandas as pd
import numpy as np

import IMP
import IMP.bff

In [37]:
def node_class_factory(
    class_name: str,
    parent: str,
    parent_kwargs: dict,
    input_attributes: typing.List[str],
    output_attributes: typing.List[str]
):
    """
    Class factory for ChiNet node classes.
    
    Parameters
    ----------
    class_name : str
        name of the produced class, e.g., `CnLifetime`
    parent : str
        class that is wrapped as ChiNet node, e.g., `IMP.bff.DecayLifetimeHandler`
    parent_kwargs : dict
        kwargs used to initialize the parent class
    input_attributes : List[str]
        list of str that refer to attributes of the parent class 
        these attributes are exposed as inputs
    output_attributes : List[str]
        list of str that refer to attributes of the parent class 
        these attributes are exposed as outputs
    
    Returns
    -------
    class
        a child class of `IMP.bff.CnNode`

    """

    class CB(IMP.bff.CnNodeCallback):

        def __init__(self, *args, **kwargs):
            super(CB, self).__init__(*args, **kwargs)

        def run(self, inputs, outputs):
            pass

    def init(obj, *args, **kwargs):
        super(obj.__class__, obj).__init__(*args, **kwargs)
        cl_name = obj.parent.split(".")[-1]
        cl = getattr(IMP.bff, cl_name)
        obj.cls = cl(**obj.parent_kwargs)

        obj.cb = CB()
        obj.set_callback(obj.cb)
        # Method for CB class
        def _in2out(inputs, outputs):
            for k in obj.input_attributes:
                setattr(obj.cls, k, inputs[k].value)
            for k in obj.output_attributes:
                v = getattr(obj.cls, k)
                outputs[k].value = v
        obj.cb.run = _in2out

        # Make Ports of CnNode
        for k in obj.input_attributes:
            v = obj.parent_kwargs[k]
            p = IMP.bff.CnPort(v, is_output=False, name=k)
            obj.add_input_port(p)
        for k in obj.output_attributes:
            v = obj.parent_kwargs[k]
            p = IMP.bff.CnPort(v, is_output=True, name=k)
            obj.add_output_port(p)
        
    attr_template = {
        "parent_kwargs" : {},
        "input_attributes": [],
        "output_attributes": [],
        "parent": "IMP.bff.None",
        "__init__": init
    }
    
    attr = {**attr_template, **cls_dict[class_name]}
    return type(class_name, (IMP.bff.CnNode, ), attr)

In [43]:
cls_dict = {
    "CnLifetime": {
        "parent_kwargs" : {
            'lifetime_spectrum': [0.012, 4.0, 2.2, 2.0],
            'abs_lifetime_spectrum': True,
            'use_amplitude_threshold': True,
            'amplitude_threshold': 0.0
        },
        "input_attributes": [
            "amplitude_threshold",
            "lifetime_spectrum"
        ],
        "output_attributes": [
            "lifetime_spectrum"
        ],
        "parent": "IMP.bff.DecayLifetimeHandler"
    },
    "CnDecay": {
        "parent_kwargs" : {
            'x': [], 'y': []
        },
        "input_attributes": ["x", "y"],
        "output_attributes": ["x", "y"],
        "parent": "IMP.bff.DecayCurve"
    },
    "CnDecayConvolution": {
        "parent_kwargs" : {
            'x': [], 'y': []
        },
        "input_attributes": ["x", "y"],
        "output_attributes": ["x", "y"],
        "parent": "IMP.bff.DecayCurve"
    }    
}

In [44]:
cls_ = dict()
for class_name in cls_dict:
    cls_[class_name] = node_class_factory(class_name, **cls_dict[class_name])
print(cls_)
locals().update(cls_)

{'CnLifetime': <class '__main__.CnLifetime'>, 'CnDecay': <class '__main__.CnDecay'>}


In [45]:
n_lt1 = CnLifetime()
n_decay = CnDecay()

In [46]:
dt = 0.0141 # time resolution (bin width of decay histogram)

fns = {
    "lintable": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_whitelight.txt"),
    "irf": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_Prompt.txt"),
    "donor": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_Donor.txt"),
    "fret": IMP.bff.get_example_path("spectroscopy/hgbp1/eTCSPC_FRET.txt")
}

data = dict()
for key in fns:
    df = pd.read_csv(fns[key], skiprows=6, sep='\t')
    data[key] = IMP.bff.DecayCurve(x=df['Chan'] * dt, y=df['Data'])

__main__.CnLifetime

In [15]:
n_decay.inputs['y'].value = 10 * [0.0]

In [16]:
n_decay.evaluate()

In [17]:
n_decay.outputs['x'].value

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [18]:
n_lt1.evaluate()

In [19]:
n_lt1.outputs["lifetime_spectrum"].value

array([0.012, 4.   , 2.2  , 2.   ])

In [15]:
settings = {
    "background_settings": {
        "start": 0, "stop": -1,
        "constant_offset": 2.,
        "pattern": None,
        "pattern_fraction": 0.0,
        "active": True
    },
    "pile_up_settings": {
        "active": False
    },
    "convolution_settings": {
        "start": 100, "stop": 3800,
        "instrument_response_function": data["irf"],
        "convolution_method": IMP.bff.DecayConvolution.FAST,
        "excitation_period": 100,
        "irf_shift_channels": 0.0,
        "irf_background_counts": 0.0,
    },
    "scale_settings": {
        "start": 0, "stop": -1,
        "constant_background": 0.0,
        "active": True,
        "blank_outside": True
    },
    "score_settings": {
        "score_type": "poisson",
        "start": 0, "stop": -1    
    }
}